<a href="https://colab.research.google.com/github/XTRA26/MiXLab/blob/master/SimDeblur_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <p align=center>`SimDeblur`</p>
A video deblurring demo with SimDeblur.

In [ ]:
!pwd

In [ ]:
!git clone https://github.com/ljzycmd/SimDeblur.git

In [ ]:
%cd SimDeblur
!bash Install.sh

Download the checkpoint and video demo

In [ ]:
# download the checkpoint of DBN
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1r3c1moyJC88yoSnyWA5zirMB_yvTpMXi' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1r3c1moyJC88yoSnyWA5zirMB_yvTpMXi" -O ./demo/dbn_ckpt.pth && rm -rf /tmp/cookies.txt

In [ ]:
# download the demo video for video deblurring
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1YJLeaQNQv8Xqw8isVArtxrwGjVzSp6gc' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1YJLeaQNQv8Xqw8isVArtxrwGjVzSp6gc" -O ./datasets/dvd_books.zip && rm -rf /tmp/cookies.txt

In [ ]:
# unzip the video frames
%cd datasets
#
!unzip dvd_books.zip
#
%cd ..

1. Import the packages

In [ ]:
import torch
from easydict import EasyDict as edict

2. Build a DBN model

In [ ]:
from simdeblur.model import build_backbone

model_cfg = edict({
    "name": "DBN",
    "num_frames": 5,
    "in_channels": 3,
    "inner_channels": 64
})

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = build_backbone(model_cfg)
ckpt = torch.load("./demo/dbn_ckpt.pth")
model_ckpt = ckpt["model"]
model_ckpt = {k[7:]: v for k, v in model_ckpt.items()}

model.load_state_dict(model_ckpt)
model = model.to(device)

3. Construct the test video data

In [ ]:
from simdeblur.dataset.frames_folder import FramesFolder

data_config = edict({
    "root_input": "./datasets/input",
    "num_frames": 5,
    "overlapping": True,
    "sampling": "n_c"
})

frames_data = FramesFolder(data_config)
frames_dataloader = torch.utils.data.DataLoader(frames_data, 1)

4. Run the inference

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(18, 12))
model.eval()
with torch.no_grad():
    # you can change the index to deblur different frames. 
    batch_data = frames_data[1]
    out = model(batch_data["input_frames"].unsqueeze(0).cuda())
    out = out.clamp(0, 1)
    plt.subplot(1, 2, 1)
    plt.title("Center blurry frame")
    plt.imshow(batch_data["input_frames"][2].permute(1, 2, 0).numpy())
    plt.subplot(1, 2, 2)
    plt.title("Latent frame")
    plt.imshow(out[0].cpu().permute(1, 2, 0).numpy())